# Load images using loader.py and perform CNN classification

@author: pawel@kasprowski.pl

In [8]:
import numpy as np
import loader

from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, cohen_kappa_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import class_weight
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Sequential

## Load data

In [9]:
samples,labels,_ = loader.load_img("radio_img")

print("shape = {}".format(samples.shape))

#add the fourth dimension (color)
samples = np.expand_dims(samples, axis=3)

print("shape = {}".format(samples.shape))
inputShape = (samples.shape[1],samples.shape[2],samples.shape[3])
print("inputShape = {}".format(inputShape))


shape = (611, 108, 192)
shape = (611, 108, 192, 1)
inputShape = (108, 192, 1)


In [10]:
#weights
class_weights = class_weight.compute_class_weight('balanced',classes=np.unique(labels),y=labels)
d_class_weights = dict(enumerate(class_weights))
print("weights {}".format(d_class_weights))

#one-hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
classesNum= labels.shape[1]
print ("Classes: {}".format(classesNum))


weights {0: 1.64247311827957, 1: 1.1252302025782688, 2: 0.6655773420479303}
Classes: 3


## Create and compile the CNN model

In [11]:
def cnn_model(inputShape,numClasses):
    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding="same",input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), padding="same",input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("sigmoid"))
    model.add(Dense(256))
    model.add(Activation("sigmoid"))
    model.add(Dense(numClasses))
    model.add(Activation("softmax"))

    loss='categorical_crossentropy'    
    model.compile(loss=loss, optimizer="adam",metrics=['accuracy'])
    return model
model = cnn_model(inputShape,classesNum)

In [12]:
#split to training and test
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels, test_size=0.25, random_state=42)

In [14]:
## callbacks
callback1 = ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5', save_best_only=True, verbose=1)
callback2 = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

EPOCHS=20
BATCH=50
model.fit(trainSamples, trainLabels, batch_size=BATCH, epochs=EPOCHS,class_weight=d_class_weights,verbose=1,
              callbacks = [callback1,callback2],
              validation_data=(testSamples,testLabels))
    
cnnResults = model.predict(testSamples)

print(confusion_matrix(testLabels.argmax(axis=1), cnnResults.argmax(axis=1)))
print(classification_report(testLabels.argmax(axis=1), cnnResults.argmax(axis=1)))
cnnAcc = accuracy_score(testLabels.argmax(axis=1), cnnResults.argmax(axis=1))
print("Accuracy CNN: {:.2f}".format(cnnAcc))
print("Cohen's Kappa {:.2f}".format(cohen_kappa_score(testLabels.argmax(axis=1), cnnResults.argmax(axis=1))))

Epoch 1/20
10/10 [==============================] - ETA: 0s - loss: 0.0191 - accuracy: 0.9978
Epoch 00001: val_loss improved from inf to 1.03415, saving model to model.01-1.03.h5
10/10 [==============================] - 3s 346ms/step - loss: 0.0191 - accuracy: 0.9978 - val_loss: 1.0342 - val_accuracy: 0.6732
Epoch 2/20
10/10 [==============================] - ETA: 0s - loss: 0.0166 - accuracy: 0.9978
Epoch 00002: val_loss did not improve from 1.03415
10/10 [==============================] - 3s 327ms/step - loss: 0.0166 - accuracy: 0.9978 - val_loss: 1.0532 - val_accuracy: 0.6732
Epoch 3/20
10/10 [==============================] - ETA: 0s - loss: 0.0142 - accuracy: 1.0000
Epoch 00003: val_loss did not improve from 1.03415
10/10 [==============================] - 3s 343ms/step - loss: 0.0142 - accuracy: 1.0000 - val_loss: 1.0852 - val_accuracy: 0.6863
Epoch 4/20
10/10 [==============================] - ETA: 0s - loss: 0.0128 - accuracy: 1.0000
Epoch 00004: val_loss did not improve from 1